In [1]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, r2_score

# --- CONFIGURATION ---
INPUT_FILE = "MASTER_CAR_DATASET.csv"
MODEL_FILE = "price_model.pkl"

# 1. Load the Fixed Master Dataset
print("Loading Master Dataset...")
df = pd.read_csv(INPUT_FILE)

# 2. Filter & Prepare Data
# We select the columns that actually determine price
features = ['title_version', 'model_year', 'mileage', 'engine', 'transmission', 'fuel', 'inspection_score']
target = 'price'

# Drop any rows that still have empty values (Safety First)
df = df.dropna(subset=features + [target])

X = df[features]
y = df[target]

print(f"Training on {len(df)} cars...")

# 3. The "Smart" Pipeline
# This allows us to feed raw text and categories directly into the AI

preprocessor = ColumnTransformer(
    transformers=[
        # A. Text: Learns that "Civic" > "City" and "Turbo" > "Simple"
        ('title', TfidfVectorizer(max_features=100, stop_words='english'), 'title_version'),
        
        # B. Categories: Handles Manual/Auto and Petrol/Diesel
        ('cats', OneHotEncoder(handle_unknown='ignore'), ['transmission', 'fuel']),
    ],
    # C. Numbers: Year, Mileage, Engine, and YOUR SCORE pass through as-is
    remainder='passthrough' 
)

# 4. Create the Model
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=300, random_state=42))
])

# 5. Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training the Final Price Model (this takes ~30 seconds)...")
model.fit(X_train, y_train)

# 6. Evaluation
print("Evaluating Accuracy...")
predictions = model.predict(X_test)
r2 = r2_score(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)

print(f"\n--- FINAL REPORT CARD ---")
print(f"Model Confidence (R2): {r2:.2%} (Target: >80%)")
print(f"Average Error: +/- PKR {int(mae):,}")

# 7. Save the Brain (THE MOST IMPORTANT STEP)
joblib.dump(model, MODEL_FILE)
print(f"SUCCESS: Model saved to {MODEL_FILE}")

# --- 8. THE "MAGIC" DEMONSTRATION ---
# This proves your FYP works. We predict the SAME car with DIFFERENT scores.

print("\n--- THE DEMONSTRATION ---")
print("Scenario: Comparing two identical 2020 Honda Civics.")

# Car A: The "Good" One (Score 9.5)
car_good = pd.DataFrame({
    'title_version': ['Honda Civic Oriel 1.8 i-VTEC CVT 2020'],
    'model_year': [2020],
    'mileage': [50000],
    'engine': [1800],
    'transmission': ['Automatic'],
    'fuel': ['Petrol'],
    'inspection_score': [9.5] # HIGH SCORE
})

# Car B: The "Bad" One (Score 6.0)
car_bad = pd.DataFrame({
    'title_version': ['Honda Civic Oriel 1.8 i-VTEC CVT 2020'],
    'model_year': [2020],
    'mileage': [50000],
    'engine': [1800],
    'transmission': ['Automatic'],
    'fuel': ['Petrol'],
    'inspection_score': [6.0] # LOW SCORE
})

price_good = model.predict(car_good)[0]
price_bad = model.predict(car_bad)[0]
diff = price_good - price_bad

print(f"Price with 9.5 Score: PKR {int(price_good):,}")
print(f"Price with 6.0 Score: PKR {int(price_bad):,}")
print(f"VALUE DIFFERENCE DETECTED: PKR {int(diff):,}")

if diff > 0:
    print("\nCONCLUSION: The AI successfully learned that Condition affects Price!")
else:
    print("\nNOTE: The AI is confused. We might need more data.")

Loading Master Dataset...
Training on 2432 cars...
Training the Final Price Model (this takes ~30 seconds)...
Evaluating Accuracy...

--- FINAL REPORT CARD ---
Model Confidence (R2): 86.17% (Target: >80%)
Average Error: +/- PKR 477,057
SUCCESS: Model saved to price_model.pkl

--- THE DEMONSTRATION ---
Scenario: Comparing two identical 2020 Honda Civics.
Price with 9.5 Score: PKR 6,170,400
Price with 6.0 Score: PKR 6,129,750
VALUE DIFFERENCE DETECTED: PKR 40,650

CONCLUSION: The AI successfully learned that Condition affects Price!
